In [37]:
from kspotter import *

In [39]:
simil = KNeighborsSpotter(k=10)

In [ ]:
simil.train()

In [35]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import trcain_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.tree import DecisionTreeRegressor

california_housing = fetch_california_housing(as_frame=True)

training = california_housing.frame[california_housing.feature_names]
target = california_housing.frame[california_housing.target_names]

X_train, X_test, y_train, y_test = train_test_split(training, target, random_state=1234, test_size=0.20)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, random_state=1234, test_size=0.20)

In [14]:
alphas = np.logspace(-3, 1, num=30)
model = make_pipeline(StandardScaler(), 
                      RidgeCV(alphas=alphas))

tree_one = DecisionTreeRegressor(max_depth=3, random_state=0)
tree_two = DecisionTreeRegressor(max_depth=10, random_state=0)

model = make_pipeline(StandardScaler(), 
                      RidgeCV(alphas=alphas))
cv_results = cross_validate(
    model, california_housing.data, california_housing.target,
    return_estimator=True, n_jobs=2)

In [16]:
california_housing.frame.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
